# To -do
* calculate population counts by dividing measures / 100_000 then * population counts
* plot incidence
* clarify about CVD

In [1]:
import pandas as pd
import numpy as np
from vivarium import Artifact
import os
from pathlib import Path
from db_queries import get_outputs, get_ids, get_model_results, get_population
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import gbd_mapping
from matplotlib.backends.backend_pdf import PdfPages
pd.set_option('use_inf_as_na', True)
sns.set(context = 'paper', style='whitegrid', font_scale=1.8, rc = {'axes.spines.right':False, 'axes.spines.top': False, 'figure.figsize':(12.7,8.6)}, palette='Set1')


## Change variable save_path to local directory file path for file results



In [2]:
mapping_dict_data = {'cause': {'acute_myocardial_infarction':'ischemic_heart_disease','post_myocardial_infarction':'ischemic_heart_disease','acute_ischemic_stroke':'ischemic_stroke','post_ischemic_stroke':'ischemic_stroke', 'ischemic_stroke':'ischemic_stroke', 'chronic_kidney_disease':'chronic_kidney_disease', 'ischemic_heart_disease':'ischemic_heart_disease', 'diabetes_mellitus':'diabetes_mellitus', 'other_causes':'other_causes', 'albuminuria':'chronic_kidney_disease', 'stage_iii_chronic_kidney_disease':'chronic_kidney_disease','stage_iv_chronic_kidney_disease':'chronic_kidney_disease','stage_v_chronic_kidney_disease':'chronic_kidney_disease','moderate_diabetes_mellitus':'diabetes_mellitus','severe_diabetes_mellitus':'diabetes_mellitus', 'chronic_kidney_disease':'chronic_kidney_disease', 'CVD':'CVD'},
                    'cause_client': {'ischemic_heart_disease':'Ischemic heart disease', 'ischemic_stroke':'Ischemic stroke', 'chronic_kidney_disease':'Chronic kidney disease', 'diabetes_mellitus':'Diabetes mellitus', 'other_causes':'Other causes', 'CVD':'CVD', 'all_cause_mortality':'All-cause mortality'},
                    'sequela':{378:'acute_myocardial_infarction', 379:'acute_myocardial_infarction',380:'post_myocardial_infarction', 381:'post_myocardial_infarction', 382:'post_myocardial_infarction', 383:'post_myocardial_infarction', 384:'post_myocardial_infarction', 385:'post_myocardial_infarction', 953:'post_myocardial_infarction', 1040:'post_myocardial_infarction', 5726:'post_myocardial_infarction', 386: 'acute_ischemic_stroke', 387: 'acute_ischemic_stroke', 388: 'acute_ischemic_stroke', 389: 'acute_ischemic_stroke', 390: 'acute_ischemic_stroke', 391:'post_ischemic_stroke', 392: 'post_ischemic_stroke', 393:'post_ischemic_stroke', 394:'post_ischemic_stroke', 395:'post_ischemic_stroke', 946:'post_ischemic_stroke'},
                    'location': {'china':'China', 'brazil':'Brazil', 'france':'France', 'italy':'Italy', 'russian_federation':'Russia', 'spain':'Spain', 'All locations':'All locations'},
                    'sex':{'female':'female','male':'male', 'both_sexes_combined':'both sexes'},
                    'scenario': {'baseline':'Business as Usual', 'guideline':'Intervention 1 (multiple pills)', 'guideline_and_new_treatment':'Intervention 2 (FDC)'},
                    'measure':{'death':'deaths','ylds':'ylds','ylls':'ylls'},
                    'measure_client':{'deaths':'Deaths', 'incidence':'Incidence'},
                    'model_7_results':{'brazil':'brazil/2020_04_07_18_51_12', 'china':'china/2020_04_07_18_54_07','france':'france/2020_04_07_18_51_32', 'italy':'italy/2020_04_07_18_55_54', 'russian_federation':'russian_federation/2020_04_07_18_51_57', 'spain':'spain/2020_04_07_18_52_09'}}

locations = ['brazil', 'china', 'france', 'italy','russian_federation', 'spain']
measures = ['deaths', 'person_time','population','ylds','ylls', 'prevalence', 'incidence']
save_path_j = '/home/j/Project/simulation_science/zenon/result/final_results'

## Zenon Model 7 results



In [3]:
path_template = Path('/share/costeffectiveness/results/vivarium_csu_zenon/v7.2_intervention/')

def get_model_outputs_sequela(locations, measure):
    outcomes = pd.DataFrame([])
    try:   
        for location in locations:
            country_loc = mapping_dict_data['model_7_results'][location]
            outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/{measure}.hdf'))
            outcome['location'] = f'{location}'
            if measure != 'transition_count':
                outcome['measure'] = f'{measure}'
            outcomes = outcomes.append(outcome)
        return outcomes
            
    except:
        print(f'{measure} is not a measure in the outputs')

def get_model_outputs_cause(locations, measure):
    outcomes = pd.DataFrame([])
    try:   
    
        for location in locations:
            country_loc = mapping_dict_data['model_7_results'][location]
            outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/{measure}.hdf'))
            outcome['location'] = f'{location}'
        
            if measure != 'transition_count':
                outcome['measure'] = f'{measure}'
            outcomes = outcomes.append(outcome)
            outcomes['cause'] = outcomes.cause.map(mapping_dict_data['cause'])
            outcomes = outcomes.dropna()
#         outcomes = outcomes.groupby(['age_group','sex','year','cause','input_draw','location', 'scenario']).value.sum().reset_index()

        return outcomes
            
    except:
            print(f'{measure} is not a measure in the outputs')



In [4]:
def get_pops(locations):
    outcomes = pd.DataFrame([])
    for country in locations:
        country_loc = mapping_dict_data['model_7_results'][country]
        outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/population.hdf'))
        outcome['location'] = f'{country}'
        outcomes = outcomes.append(outcome)
    return outcomes

def get_person_time(locations):
    outcomes = pd.DataFrame([])
    for country in locations:
        country_loc = mapping_dict_data['model_7_results'][country]
        outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/person_time.hdf'))
        outcome['location'] = f'{country}'
        outcomes = outcomes.append(outcome)
#     outcomes = outcomes.groupby(['age_group','sex','year','input_draw', 'location', 'scenario']).value.sum().reset_index()
    return outcomes

def get_person_time_misc(locations):
    outcomes = pd.DataFrame([])
    for country in locations:
        country_loc = mapping_dict_data['model_7_results'][country]
        outcome = pd.read_hdf(str(path_template / f'{country_loc}/count_data/miscellaneous_person_time.hdf'))
        outcome['location'] = f'{country}'
        outcomes = outcomes.append(outcome)
    return outcomes

In [5]:
def get_transition(locations):
    outcomes = pd.DataFrame([])
    for country in locations:
        country_loc = mapping_dict_data['model_7_results'][country]
        df = pd.read_hdf(str(path_template / f'{country_loc}/count_data/transition_count.hdf'))
        df['location'] = f'{country}'
        outcomes = outcomes.append(df)
    return outcomes

def get_transition_susceptible_to_acute(locations):
    transitions = get_model_outputs_sequela(locations, 'transition_count')
    transitions = transitions[transitions.measure.str.startswith('susceptible')]
    transitions['susceptible'] = transitions.measure.str.extract('((?<=to_).*?(?=_to))',expand = False)
    transitions['cause'] = transitions.measure.str.extract('((?<=to_).*?(?=_event_count))',expand = False)
    transitions['cause'] = transitions.cause.str.extract('((?<=_to_).*)', expand = False)
    transitions = transitions.drop('measure', axis = 1)
    return transitions

def get_transition_post_to_acute(locations):    
    transitions = get_model_outputs_sequela(locations, 'transition_count')
    transitions = transitions[transitions.measure.str.startswith('post')]
    transitions['susceptible'] = transitions.measure.str.extract('(.*?(?=_to))',expand = False)
    transitions['cause'] = transitions.measure.str.extract('((?<=to_).*?(?=_event_count))',expand = False)
    transitions = transitions.drop('measure', axis = 1)
    return transitions

In [6]:
susceptible_to_acute = get_transition_susceptible_to_acute(locations)
post_to_acute = get_transition_post_to_acute(locations)

In [4]:
def collapse_frame(df, column):
    nonidcols = ['value']
    nonidcols.extend(column)
    return(df.groupby([x for x in df.columns if x not in nonidcols])['value'].sum().reset_index())

# Rate-space results
## Added subpopulation to cause_measures function and diabetes + hypertensive subpop. to incidence and cause measures function

In [8]:
ylds = get_model_outputs_cause(locations, 'ylds')
ylds['measure'] = 'ylds'
ylls = get_model_outputs_cause(locations, 'ylls')
ylls['measure'] = 'ylls'
deaths = get_model_outputs_cause(locations, 'deaths')
deaths['measure'] = 'deaths'
state = get_model_outputs_cause(locations, 'state_person_time')
state['measure'] = 'prevalence'
person_time = get_person_time(locations)


## Get rate-space disease outcomes per location

In [146]:
def get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include = None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True):
    transitions = pd.concat([susceptible_to_acute, post_to_acute])
    transitions['subpopulation'] = 'total_population'
    state = get_model_outputs_sequela(locations, 'state_person_time')
    state['subpopulation'] = 'total_population'
    transitions = transitions[(transitions.age_group != '30_to_34') & (transitions.age_group != '35_to_39') & (transitions.age_group != '85_to_89') & (transitions.age_group != '90_to_94') & (transitions.age_group != '95_plus')]
    #added this later on
    state = state[(state.age_group != '30_to_34') & (state.age_group != '35_to_39') & (state.age_group != '85_to_89') & (state.age_group != '90_to_94') & (state.age_group != '95_plus')]
    state = state[(state.cause.str.contains('susceptible')) | (state.cause.str.contains('post'))]
    state['susceptible'] = state.cause.str.replace('susceptible_to_', '')
    if causes_to_include != None:
        transitions = transitions[transitions.susceptible.isin(causes_to_include)]
        state = state[state.susceptible.isin(causes_to_include)]
    state = state.drop(['cause', 'measure'], axis = 1)
    if not risk_strat:
        state = collapse_frame(state, ['acs','fpg', 'sbp', 'ldl'])
        transitions = collapse_frame(transitions, ['acs','fpg', 'sbp', 'ldl'])
    if not age_strat_all_ages:
        transitions = collapse_frame(transitions, ['age_group'])
        transitions['age_group'] = 'All_ages'
        state = collapse_frame(state, ['age_group'])
        state['age_group'] = 'All_ages'
    if not age_strat_75plus:
        transitions_75 = transitions[(transitions.age_group == '75_to_79') | (transitions.age_group == '80_to_84')]
        transitions = collapse_frame(transitions_75, ['age_group'])
        transitions['age_group'] = '75_plus'
        state_75 = state[(state.age_group == '75_to_79') | (state.age_group == '80_to_84')]
        state = collapse_frame(state_75, ['age_group'])
        state['age_group'] = '75_plus'
    if not age_strat_40_74:
        transitions_40_74 = transitions[transitions.age_group.isin(['40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74'])]
        transitions = collapse_frame(transitions_40_74, ['age_group'])
        transitions['age_group'] = '40_to_74'
        state_40_74 = state[state.age_group.isin(['40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74'])]
        state = collapse_frame(state_40_74, ['age_group'])
        state['age_group'] = '40_to_74'
    if not sex_strat:
        transitions = collapse_frame(transitions, ['sex'])
        transitions['sex'] = 'both_sexes_combined'
        state = collapse_frame(state, ['sex'])
        state['sex'] = 'both_sexes_combined'
    if not cvd_strat:
        transitions = collapse_frame(transitions, ['cause'])
        transitions['susceptible'] = 'CVD'
        state = collapse_frame(state, ['susceptible'])
        state['susceptible'] = 'CVD'
    if not loc_strat:
        transitions = collapse_frame(transitions, ['location'])
        transitions['location'] = 'All locations'
        state = collapse_frame(state, ['location'])
        state['location'] = 'All locations'
    if not diabetes_strat:
        transitions_dm = transitions[(transitions.fpg=='high')]
        transitions = collapse_frame(transitions_dm, ['acs','fpg', 'sbp', 'ldl'])
        transitions['subpopulation'] = 'diabetic'
        state_dm = state[(state.fpg=='high')]
        state = collapse_frame(state_dm, ['acs','fpg', 'sbp', 'ldl'])
        state['subpopulation'] = 'diabetic'
    if not htn_strat:
        transitions_htn = transitions[(transitions.sbp=='high')]
        transitions = collapse_frame(transitions, ['acs','fpg', 'sbp', 'ldl'])
        transitions['subpopulation'] = 'hypertensive'
        state_htn = state[(state.sbp=='high')]
        state = collapse_frame(state_htn, ['acs','fpg', 'sbp', 'ldl'])
        state['subpopulation'] = 'hypertensive' 
    if not high_ldl_strat:
        transitions_ldl = transitions[(transitions.ldl=='high')]
        transitions = collapse_frame(transitions_ldl, ['acs','fpg', 'sbp', 'ldl'])
        transitions['subpopulation'] = 'LDL-c > 3'
        state_ldl = state[(state.ldl=='high')]
        state = collapse_frame(state_ldl, ['acs','fpg', 'sbp', 'ldl'])
        state['subpopulation'] = 'LDL-c > 3'
    if not year_strat:
        transitions = collapse_frame(transitions, ['year'])
        transitions['year'] = 'All years'
        state = collapse_frame(state, ['year'])
        state['year'] = 'All years'
    if not final_cause_strat:
        transitions = collapse_frame(transitions, ['cause'])
        transitions = transitions.set_index([x for x in state.columns if x != 'value']).value
    else:
        transitions = transitions.set_index([x for x in state.columns if x != 'value'] + ['cause']).value
        #added this in to test transitions
    state = state.set_index([x for x in state.columns if x != 'value']).value
    incidence = (transitions/state * 100_000).reset_index()
    incidence['measure'] = 'incidence'
    if 'cause' not in incidence.columns:
        incidence.rename(columns = {'susceptible':'cause'}, inplace = True)
    incidence['cause'] = incidence.cause.map(mapping_dict_data['cause'])

    return incidence

In [147]:
# htn 
incidence_htn_all_ages = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_all_ages_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None,  age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_40to74 = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_40to74_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None,  age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_75plus = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_75plus_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_htn_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = False, diabetes_strat = True, high_ldl_strat=True, year_strat=True)


#diabetes
incidence_db_all_ages = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_all_ages_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_40to74 = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_40to74_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_75plus = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_75plus_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)
incidence_db_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = False, high_ldl_strat=True, year_strat=True)

#high_ldl
incidence_ldl_all_ages = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_all_ages_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_40to74 = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_40to74_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_75plus = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_75plus_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)
incidence_ldl_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = True, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=False, year_strat=True)

#total
incidence_total_all_ages = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_all_ages_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_40to74 = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_40to74_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_75plus = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_75plus_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)
incidence_total_both = get_incidence(susceptible_to_acute, post_to_acute, locations, risk_strat = False, final_cause_strat = False, cvd_strat=True, causes_to_include=None, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, loc_strat=True, htn_strat = True, diabetes_strat = True, high_ldl_strat=True, year_strat=True)


In [148]:
incidence_measures = pd.concat([incidence_htn_all_ages, incidence_htn_all_ages_both,
incidence_htn_40to74,
incidence_htn_40to74_both,
incidence_htn_75plus,
incidence_htn_75plus_both, incidence_htn,
incidence_htn_both,
incidence_db_all_ages,
incidence_db_all_ages_both,
incidence_db_40to74,
incidence_db_40to74_both,
incidence_db_75plus,
incidence_db_75plus_both,
incidence_db,
incidence_db_both,
incidence_ldl_all_ages,
incidence_ldl_all_ages_both,
incidence_ldl_40to74,
incidence_ldl_40to74_both,
incidence_ldl_75plus,
incidence_ldl_75plus_both,
incidence_ldl,
incidence_ldl_both,
incidence_total_all_ages,
incidence_total_all_ages_both,
incidence_total_40to74,
incidence_total_40to74_both,
incidence_total_75plus,
incidence_total_75plus_both, incidence_total, incidence_total_both])


/share/code/santonis/miniconda/envs/validation/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [151]:
incidence_measures.to_csv('incidence_measures.csv', index=False)

In [8]:
incidence_measures = pd.read_csv('incidence_measures.csv')

In [90]:
def get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True):

    measures = pd.concat([ylds, ylls, deaths, state])
    person_time = person_time.drop(columns=['measure'])
    measures['subpopulation'] = 'total_population'
    person_time['subpopulation'] = 'total_population'

    measures = measures[(measures.age_group != '30_to_34') & (measures.age_group != '35_to_39') & (measures.age_group != '85_to_89') & (measures.age_group != '90_to_94') & (measures.age_group != '95_plus')]
    #added this later on
    person_time = person_time[(person_time.age_group != '30_to_34') & (person_time.age_group != '35_to_39') & (person_time.age_group != '85_to_89') & (person_time.age_group != '90_to_94') & (person_time.age_group != '95_plus')]

    if causes_to_include != None:
        measures = measures[measures.cause.isin(causes_to_include)]

    if not risk_strat:
        measures = collapse_frame(measures, ['acs','fpg', 'sbp', 'ldl'])
        person_time = collapse_frame(person_time, ['acs','fpg', 'sbp', 'ldl'])
        
    if not age_strat_all_ages:
        measures = collapse_frame(measures, ['age_group'])
        measures['age_group'] = 'All_ages'
        person_time = collapse_frame(person_time, ['age_group'])
        person_time['age_group'] = 'All_ages'
    if not age_strat_75plus:
        measures_75 = measures[(measures.age_group == '75_to_79') | (measures.age_group == '80_to_84')]
        measures = collapse_frame(measures_75, ['age_group'])
        measures['age_group'] = '75_plus'
        pt_75 = person_time[(person_time.age_group == '75_to_79') | (person_time.age_group == '80_to_84')]
        person_time = collapse_frame(pt_75, ['age_group'])
        person_time['age_group'] = '75_plus'
    if not age_strat_40_74:
        measures_40_74 = measures[measures.age_group.isin(['40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74'])]
        measures = collapse_frame(measures_40_74, ['age_group'])
        measures['age_group'] = '40_to_74'
        pt_40_74 = person_time[person_time.age_group.isin(['40_to_44', '45_to_49', '50_to_54', '55_to_59', '60_to_64', '65_to_69', '70_to_74'])]
        person_time = collapse_frame(pt_40_74, ['age_group'])
        person_time['age_group'] = '40_to_74'
    if not sex_strat:
        measures = collapse_frame(measures, ['sex'])
        measures['sex'] = 'both_sexes_combined'
        person_time = collapse_frame(person_time, ['sex'])
        person_time['sex'] = 'both_sexes_combined'
    if not cvd_strat:
        measures = collapse_frame(measures, ['cause'])
        measures['cause'] = 'CVD'

    if not loc_strat:
        measures = collapse_frame(measures, ['location'])
        measures['location'] = 'All locations'
        person_time = collapse_frame(person_time, ['location'])
        person_time['location'] = 'All locations'
    if not diabetes_strat:
        measures_dm = measures[(measures.fpg=='high')]
        measures = collapse_frame(measures_dm, ['acs','fpg', 'sbp', 'ldl'])
        measures['subpopulation'] = 'diabetic'
        person_time_dm = person_time[(person_time.fpg=='high')]
        person_time = collapse_frame(person_time_dm, ['acs','fpg', 'sbp', 'ldl'])
        person_time['subpopulation'] = 'diabetic'
    if not htn_strat:
        measures_htn = measures[(measures.sbp=='high')]
        measures = collapse_frame(measures_htn, ['acs','fpg', 'sbp', 'ldl'])
        measures['subpopulation'] = 'hypertensive'
        person_time_htn = person_time[(person_time.sbp=='high')]
        person_time = collapse_frame(person_time_htn, ['acs','fpg', 'sbp', 'ldl'])
        person_time['subpopulation'] = 'hypertensive' 
    if not high_ldl_strat:
        measures_ldl = measures[(measures.ldl=='high')]
        measures = collapse_frame(measures_ldl, ['acs','fpg', 'sbp', 'ldl'])
        measures['subpopulation'] = 'LDL-c > 3'
        person_time_ldl = person_time[(person_time.ldl=='high')]
        person_time = collapse_frame(person_time_ldl, ['acs','fpg', 'sbp', 'ldl'])
        person_time['subpopulation'] = 'LDL-c > 3'
    if not year_strat:
        measures = collapse_frame(measures, ['year'])
        measures['year'] = 'All years'
        person_time = collapse_frame(person_time, ['year'])
        person_time['year'] = 'All years'
    if not acmr_strat:
        measures_acmr = measures[(measures.measure == 'deaths')]
        measures = collapse_frame(measures_acmr, ['cause'])
        measures['cause'] = 'all_cause_mortality'
    if not daly_strat:
        measures_dalys = measures[((measures.measure == 'ylls') | (measures.measure == 'ylds'))]
        measures = collapse_frame(measures_dalys, ['measure'])
        measures['measure'] = 'dalys'

    measures = measures.set_index([x for x in person_time.columns if x != 'value'] + ['cause', 'measure']).value

    person_time = person_time.set_index([x for x in person_time.columns if x != 'value']).value
    rates = (measures / person_time * 100_000).reset_index()

    return rates

In [93]:
#htn strat
measures_htn_allages = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_allages_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_allages_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_allages_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_htn_allages_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_allages_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_htn_75plus = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_75plus_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_75plus_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_75plus_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_htn_75plus_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_75plus_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_htn_40to74 = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_40to74_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_40to74_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_40to74_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_htn_40to74_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_40to74_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_htn = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_htn_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_htn_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_htn_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=False, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

#diabetes strat
measures_db_allages = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_allages_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_allages_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_allages_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_db_allages_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_allages_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_db_75plus = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_75plus_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_75plus_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_75plus_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_db_75plus_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_75plus_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_db_40to74 = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_40to74_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_40to74_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_40to74_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_db_40to74_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_40to74_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_db = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_db_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_db_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_db_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=False, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

#High-LDL strat
measures_ldl_allages = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_allages_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_allages_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_allages_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)
measures_ldl_allages_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_allages_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)

measures_ldl_75plus = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_75plus_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_75plus_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_75plus_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)
measures_ldl_75plus_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_75plus_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)

measures_ldl_40to74 = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_40to74_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_40to74_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_40to74_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)
measures_ldl_40to74_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_40to74_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)

measures_ldl = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=True)
measures_ldl_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)
measures_ldl_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=False, daly_strat=True)
measures_ldl_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=True, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=False, year_strat=True, acmr_strat=True, daly_strat=False)

#no risk strat
measures_total_allages = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_allages_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_allages_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_allages_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_total_allages_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_allages_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=False, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_total_75plus = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_75plus_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_75plus_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_75plus_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_total_75plus_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_75plus_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=False, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_total_40to74 = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_40to74_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_40to74_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_40to74_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_total_40to74_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_40to74_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=False, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)

measures_total = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_both = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=True)
measures_total_both_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_both_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=False, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)
measures_total_acmr = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=False, daly_strat=True)
measures_total_dalys = get_cause_specific_rates(ylds, ylls, deaths, state, person_time, risk_strat=False, age_strat_all_ages=True, age_strat_75plus=True, age_strat_40_74=True, sex_strat=True, cvd_strat=True, causes_to_include=None, loc_strat=True, htn_strat=True, diabetes_strat=True, high_ldl_strat=True, year_strat=True, acmr_strat=True, daly_strat=False)


/share/code/santonis/miniconda/envs/validation/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [94]:
all_measures = pd.concat([measures_htn_allages, measures_htn_allages_both, measures_htn_allages_both_acmr, measures_htn_allages_both_dalys, measures_htn_allages_acmr, measures_htn_allages_dalys, measures_htn_75plus, measures_htn_75plus_both, measures_htn_75plus_both_acmr, measures_htn_75plus_both_dalys, measures_htn_75plus_acmr, measures_htn_75plus_dalys, measures_htn_40to74, measures_htn_40to74_both, measures_htn_40to74_both_acmr, measures_htn_40to74_both_dalys, measures_htn_40to74_acmr, measures_htn_40to74_dalys, measures_htn, measures_htn_both, measures_htn_both_acmr, measures_htn_both_dalys, measures_htn_acmr, measures_htn_dalys, measures_db_allages, measures_db_allages_both, measures_db_allages_both_acmr, measures_db_allages_both_dalys, measures_db_allages_acmr, measures_db_allages_dalys, measures_db_75plus, measures_db_75plus_both, measures_db_75plus_both_acmr, measures_db_75plus_both_dalys, measures_db_75plus_acmr, measures_db_75plus_dalys, measures_db_40to74, measures_db_40to74_both, measures_db_40to74_both_acmr, measures_db_40to74_both_dalys, measures_db_40to74_acmr, measures_db_40to74_dalys, measures_db, measures_db_both, measures_db_both_acmr, measures_db_both_dalys, measures_db_acmr, measures_db_dalys, measures_ldl_allages, measures_ldl_allages_both, measures_ldl_allages_both_acmr, measures_ldl_allages_both_dalys, measures_ldl_allages_acmr, measures_ldl_allages_dalys, measures_ldl_75plus, measures_ldl_75plus_both, measures_ldl_75plus_both_acmr, measures_ldl_75plus_both_dalys, measures_ldl_75plus_acmr, measures_ldl_75plus_dalys, measures_ldl_40to74, measures_ldl_40to74_both, measures_ldl_40to74_both_acmr, measures_ldl_40to74_both_dalys, measures_ldl_40to74_acmr, measures_ldl_40to74_dalys, measures_ldl, measures_ldl_both, measures_ldl_both_acmr, measures_ldl_both_dalys, measures_ldl_acmr, measures_ldl_dalys, measures_total_allages, measures_total_allages_both, measures_total_allages_both_acmr, measures_total_allages_both_dalys, measures_total_allages_acmr, measures_total_allages_dalys, measures_total_75plus, measures_total_75plus_both, measures_total_75plus_both_acmr, measures_total_75plus_both_dalys, measures_total_75plus_acmr, measures_total_75plus_dalys, measures_total_40to74, measures_total_40to74_both, measures_total_40to74_both_acmr, measures_total_40to74_both_dalys, measures_total_40to74_acmr, measures_total_40to74_dalys, measures_total , measures_total_both, measures_total_both_acmr, measures_total_both_dalys, measures_total_acmr , measures_total_dalys])


/share/code/santonis/miniconda/envs/validation/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [96]:
all_measures.to_csv('all_measures.csv', index=False)

In [6]:
all_measures = pd.read_csv('all_measures.csv')

In [9]:
def get_ui_outcomes(all_measures, incidence_measures):
    shared_columns = ['age_group','sex','year', 'cause', 'location', 'measure','scenario', 'subpopulation']
    df = all_measures
    df = df.append(incidence_measures, ignore_index=True)
    baseline = df[df.scenario == 'baseline']
    baseline = baseline.rename(columns = {'value':'baseline'})
    df = pd.merge(df, baseline, left_on=['age_group', 'sex', 'year', 'cause', 'input_draw', 'location', 'measure', 'subpopulation'], right_on=['age_group', 'sex', 'year', 'cause', 'input_draw', 'location', 'measure', 'subpopulation'], how='left')
    df['difference_from_baseline_count'] = df.apply(lambda row: (row['baseline'] - row['value']),axis=1)
    df = df.rename(columns={'scenario_x':'scenario'})
    df = df.drop(columns=['scenario_y', 'baseline'])
    ldl_std_baseline_diff = df.groupby(shared_columns)['difference_from_baseline_count'].agg([('std_baseline_diff_count','std')]).reset_index()
    ldl_mean_baseline_diff = df.groupby(shared_columns)['difference_from_baseline_count'].agg([('mean_baseline_diff_count','mean')]).reset_index()
    measure_mean = df.groupby(shared_columns)['value'].agg([('mean_value','mean')]).reset_index()
    measure_std = df.groupby(shared_columns)['value'].agg([('std_value','std')]).reset_index()
    ui_frames = [ldl_std_baseline_diff[['std_baseline_diff_count']],ldl_mean_baseline_diff[['mean_baseline_diff_count']],measure_mean[['age_group','sex', 'cause','year', 'location', 'measure','scenario', 'subpopulation', 'mean_value']], measure_std[['std_value']]]
    df = pd.concat(ui_frames,axis=1)
    df['upper_std_baseline_diff'] = df.apply(lambda row: (row['mean_baseline_diff_count'] + row['std_baseline_diff_count']),axis=1)
    df['upper_value'] = df.apply(lambda row: (row['mean_value'] + row['std_value']),axis=1)
    df['lower_std_baseline_diff'] = df.apply(lambda row: (row['mean_baseline_diff_count'] - row['std_baseline_diff_count']),axis=1)
    df['lower_value'] = df.apply(lambda row: (row['mean_value'] - row['std_value']),axis=1)

#     df = df.groupby(shared_columns)['difference_from_baseline_percent', 'difference_from_scenario_1_percent'].mean().reset_index()
    df = df.round(2)
    df['subpopulation'] = df.subpopulation.str.replace('high_ldl','LDL-c > 4.9')
    df['subpopulation'] = df.subpopulation.str.replace('hypertensive','Hypertension (SBP > 140 mmHg)')
    df['subpopulation'] = df.subpopulation.str.replace('diabetic','Diabetes (FPG > 7.0 mmol/l)')
    df['subpopulation'] = df.subpopulation.str.replace('total_population','Entire population')
    df['sex'] = df.sex.map(mapping_dict_data['sex'])
    df = df[(df.cause != 'chronic_kidney_disease') & (df.cause != 'diabetes_mellitus') & (df.cause != 'other_causes') & (df.measure != 'prevalence')]
    ihd = df[(df.cause == 'ischemic_heart_disease')]
    ihd['measure'] = ihd.measure.str.replace('dalys', 'DALYs due to ACS (Myocardial infarction)')
    ihd['measure'] = ihd.measure.str.replace('ylls', 'YLLs due to ACS (Myocardial infarction)')
    ihd['measure'] = ihd.measure.str.replace('ylds', 'YLDs due to ACS (Myocardial infarction)')
    ihd['measure'] = ihd.measure.str.replace('deaths', 'Mortality due to ACS (Myocardial infarction)')
    ihd['measure'] = ihd.measure.str.replace('incidence', 'Incidence of ACS (Myocardial infarction)')

    i_stroke = df[(df.cause == 'ischemic_stroke')]
    i_stroke['measure'] = i_stroke.measure.str.replace('dalys', 'DALYs due to Ischemic stroke')
    i_stroke['measure'] = i_stroke.measure.str.replace('ylls', 'YLLs due to Ischemic stroke')
    i_stroke['measure'] = i_stroke.measure.str.replace('ylds', 'YLDs due to Ischemic stroke')
    i_stroke['measure'] = i_stroke.measure.str.replace('deaths', 'Mortality due to Ischemic stroke')
    i_stroke['measure'] = i_stroke.measure.str.replace('incidence', 'Incidence of ACS (Myocardial infarction)')
    acmr = df[(df.cause == 'all_cause_mortality')]
    acmr['measure'] = acmr.measure.str.replace('deaths', 'All-cause mortality')
    df = pd.concat([ihd, i_stroke, acmr])
    df = df.drop(columns=['cause'])
#     df['cause'] = df.cause.map(mapping_dict_data['cause_client'])
    
    df['location'] = df.location.map(mapping_dict_data['location'])
    df['sex'] = df.sex.str.capitalize()
    df = df.rename(columns={'measure':'Outcome'})
    df = df[(df.age_group != '75_to_79') & (df.age_group != '80_to_84')]
    df['age_group'] = df.age_group.str.replace('_', ' ')
    df['age_group'] = df.age_group.str.replace(' plus', '+')
    df['scenario'] = df.scenario.map(mapping_dict_data['scenario'])
    df = df.rename(columns={'age_group':'Age group', 'cause':'Cause', 'sex':'Sex', 'scenario':'Scenario', 'location':'Location name', 
                            'subpopulation':'Subpopulation', 'measure':'Measure', 'year':'Year', 
                            'mean_value':'Mean estimated rate (per 100,000 person-years)', 
                            'lower_value':'Lower uncertainty interval (per 100,000 person-years)', 
                            'upper_value':'Upper uncertainty interval (per 100,000 person-years)', 
                            'mean_baseline_diff_count':'Difference from baseline- mean rate (per 100,000 person-years)',
                            'upper_std_baseline_diff':'Difference from baseline- upper uncertainty interval (per 100,000 person-years)', 
                            'lower_std_baseline_diff':'Difference from baseline- lower uncertainty interval (per 100,000 person-years)'})
    df = df[['Location name', 'Year', 'Subpopulation', 'Age group', 'Sex', 'Scenario', 'Outcome',  'Mean estimated rate (per 100,000 person-years)', 'Lower uncertainty interval (per 100,000 person-years)', 'Upper uncertainty interval (per 100,000 person-years)',  'Difference from baseline- mean rate (per 100,000 person-years)', 'Difference from baseline- lower uncertainty interval (per 100,000 person-years)', 'Difference from baseline- upper uncertainty interval (per 100,000 person-years)']]
    df['Sex'] = df.Sex.str.replace('Both sexes','Both')
    return df

disease_outcomes = get_ui_outcomes(all_measures, incidence_measures)


/share/code/santonis/miniconda/envs/validation/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/share/code/santonis/miniconda/envs/validation/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/share/code/santonis/miniconda/envs/validation/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [158]:
disease_outcomes.to_csv(str(save_path_j)+f'/tables/sanofi_zenon_disease_outcomes.csv', index=False)

In [25]:
def get_gbd_pops():
    ages = list(range(13, 21))+[30]
    numbers = 1
    pops = get_population(age_group_id=ages, single_year_age=False, location_id=[135,6,80,86, 62, 92], location_set_id=1,sex_id=[1,2,3],gbd_round_id=6, with_ui=True, decomp_step='iterative', forecasted_pop=False)
    ages = get_ids('age_group')
    locs = get_ids('location')
    sexes = get_ids('sex')
    pops_ages = pd.merge(pops,ages,left_on='age_group_id', right_on='age_group_id',how='left')
    pops_locs = pd.merge(pops_ages,locs,left_on='location_id',right_on='location_id',how='left')
    pops_sexs = pd.merge(pops_locs,sexes,left_on='sex_id',right_on='sex_id',how='left')
    pops = pops_sexs.drop(columns='location_description')
    pops_40plus = pops.groupby(['location_name','sex']).population.sum().reset_index()
    pops_40plus['age_group_name'] = 'All ages'
    pops_40to74 = pops[(pops['age_group_id'] >= 13) & (pops['age_group_id'] <= 19)]
    pops_40to74 = pops_40to74.groupby(['location_name','sex']).population.sum().reset_index()
    pops_40to74['age_group_name'] = '40 to 74'
    pops_75plus = pops[(pops['age_group_id'] == 20) | (pops['age_group_id'] == 30)]
    pops_75plus = pops_75plus.groupby(['location_name','sex']).population.sum().reset_index()
    pops_75plus['age_group_name'] = '75+'
    pops_frames = [pops,pops_40plus, pops_40to74, pops_75plus]
    pops = pd.concat(pops_frames)
#     pops['sex'] = pops.sex.str.replace('Male','male')
#     pops['sex'] = pops.sex.str.replace('Female','female')
#     pops['sex'] = pops.sex.str.replace('Both','both sexes')
    pops['location_name'] = pops.location_name.str.replace('Russian Federation', 'Russia')
    pops = pops[['location_name','sex','age_group_name','population']]

    return pops

In [26]:
gbd_pops = get_gbd_pops()

/share/code/santonis/miniconda/envs/validation/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [27]:
def get_population_counts(disease_outcomes, gbd_pops):
    df = disease_outcomes
    df['mean_rate'] = df.apply(lambda row: (row['Mean estimated rate (per 100,000 person-years)']/100_000),axis=1)
    df['lower_estimate_rate'] = df.apply(lambda row: (row['Lower uncertainty interval (per 100,000 person-years)']/100_000),axis=1)
    df['upper_estimate_rate'] = df.apply(lambda row: (row['Upper uncertainty interval (per 100,000 person-years)']/100_000),axis=1)
    df['mean_baseline_diff_rate'] = df.apply(lambda row: (row['Difference from baseline- mean rate (per 100,000 person-years)']/100_000),axis=1)
    df['lower_baseline_diff_rate'] = df.apply(lambda row: (row['Difference from baseline- lower uncertainty interval (per 100,000 person-years)']/100_000),axis=1)
    df['upper_baseline_diff_rate'] = df.apply(lambda row: (row['Difference from baseline- upper uncertainty interval (per 100,000 person-years)']/100_000),axis=1)
    df = pd.merge(df, gbd_pops, left_on=['Location name', 'Sex', 'Age group'], right_on=['location_name', 'sex', 'age_group_name'], how='left')
    df['Population counts estimate'] = df.apply(lambda row: (row['mean_rate'] * row['population']),axis=1)
    df['Population counts lower estimate'] = df.apply(lambda row: (row['lower_estimate_rate'] * row['population']),axis=1)
    df['Population counts upper estimate'] = df.apply(lambda row: (row['upper_estimate_rate'] * row['population']),axis=1)
    df['Population counts- difference from baseline estimate'] = df.apply(lambda row: (row['mean_baseline_diff_rate'] * row['population']),axis=1)
    df['Population counts- difference from baseline lower estimate'] = df.apply(lambda row: (row['lower_baseline_diff_rate'] * row['population']),axis=1)
    df['Population counts- difference from baseline upper estimate'] = df.apply(lambda row: (row['upper_baseline_diff_rate'] * row['population']),axis=1)
    df = df.drop(columns=['population', 'mean_rate', 'lower_estimate_rate', 'upper_estimate_rate', 'mean_baseline_diff_rate', 'lower_baseline_diff_rate', 'upper_baseline_diff_rate'])
    return df


In [28]:
population_counts = get_population_counts(disease_outcomes, gbd_pops)

In [31]:
population_counts.to_csv('sanofi_zenon_populationc_counts.csv', index=False)